In [1]:
from models.rnn.rnn_plain import RNN_Plain
from models.rnn.rnn_attention import RNN_Attention

from lang import load_data_int_seq
from utils import accuracy_score
%load_ext autoreload
%autoreload 2

In [2]:
output_lang, input_lang, train, X_test, y_test = load_data_int_seq()

## Plain RNN

In [3]:
algo = RNN_Plain(symbols = "+*-0123456789t", output_sequence_length = 9, encoded_seq_length = 9, num_epochs= 100, input_size=input_lang.n_words, hidden_size=512, output_size=output_lang.n_words, embedding_size = 64, calc_magnitude_on=False)
algo.train(input_lang, output_lang, train)

0m 1s (- 0m 11s) (10 10%) 2.6471
0m 1s (- 0m 7s) (20 20%) 2.4205
0m 2s (- 0m 6s) (30 30%) 2.4668
0m 3s (- 0m 5s) (40 40%) 2.3941
0m 3s (- 0m 3s) (50 50%) 2.3524
0m 4s (- 0m 2s) (60 60%) 2.3967
0m 4s (- 0m 1s) (70 70%) 2.3283
0m 4s (- 0m 1s) (80 80%) 2.3018
0m 5s (- 0m 0s) (90 90%) 2.2358
0m 5s (- 0m 0s) (100 100%) 2.2832


In [ ]:
pred = algo.infer(input_lang, output_lang, X_test[:1000])
pred[:25]

In [ ]:
accuracy_score(pred, y_test[:1000])

### Plain RNN with Magnitude loss

In [ ]:
magn_algo = RNN_Plain(symbols = "+*-0123456789t", output_sequence_length = 9, encoded_seq_length = 9, num_epochs= 5000, input_size=input_lang.n_words, hidden_size=512, output_size=output_lang.n_words, calc_magnitude_on=True)
magn_algo.train(input_lang, output_lang, train)

In [ ]:
pred = magn_algo.infer(input_lang, output_lang, X_test[:1000])
pred[:25]

In [ ]:
accuracy_score(pred, y_test[:1000])

##  Attention-based

In [ ]:
algo_attn = RNN_Attention(symbols = "+*-0123456789t", output_sequence_length = 9, encoded_seq_length = 9, num_epochs= 5000, input_size=input_lang.n_words, hidden_size=512, output_size=output_lang.n_words, calc_magnitude_on=False)
algo_attn.train(input_lang, output_lang, train)

In [ ]:
pred = algo_attn.infer(input_lang, output_lang, X_test[:1000])
pred[:25]

In [ ]:
accuracy_score(pred, y_test[:1000])

### Attention with magnitude loss

In [ ]:
algo_attn_magn = RNN_Attention(symbols = "+*-0123456789t", output_sequence_length = 9, encoded_seq_length = 9, num_epochs= 5000, input_size=input_lang.n_words, hidden_size=512, output_size=output_lang.n_words, calc_magnitude_on=True)
algo_attn_magn.train(input_lang, output_lang, train)

In [ ]:
pred = algo_attn_magn.infer(input_lang, output_lang, X_test[:1000])
pred[:25]

In [ ]:
accuracy_score(pred, y_test[:1000])